# Lesson 2: Artifact Lineage
***Key Concepts:*** *Artifacts, Artifact Stores, Metadata Stores, Versioning, Caching*

In this lesson we will learn about one of the coolest features of ML pipelines: automatic artifact versioning and tracking. This will give us tremendous insights into how exactly each of our models was created. Furthermore, it enables automatic artifact caching, such that we can switch out parts of our ML pipelines without needing to rerun any of the prior steps.

Before we dive into any of this, let's get clear on what exactly **[Artifacts](https://docs.zenml.io/core-concepts#artifact)** are. To illustrate, let us first rebuild our digits pipeline from the previous chapter:

In [ ]:
from zenml.pipelines import pipeline

from src.steps.importer import importer
from src.steps.sklearn_trainer import svc_trainer
from src.steps.evaluator import evaluator


@pipeline
def digits_pipeline(importer, trainer, evaluator):
    """Links all the steps together in a pipeline"""
    X_train, X_test, y_train, y_test = importer()
    model = trainer(X_train=X_train, y_train=y_train)
    evaluator(X_test=X_test, y_test=y_test, model=model)

The artifacts of this pipeline are simply the local variables we defined: `X_train`, `X_test`, `y_train`, `y_test`, and `model`. These make up the data that flows in and out of our steps. In fact, this data is at the core of our pipelines, and the pipeline definition above just defines which artifact is the input or output of what step.

## Pipeline Visualization with Dash

To visualize how the steps connect the different artifacts, we can view our pipeline with ZenML's [Dash](https://dash.plotly.com/introduction) integration. Run the following code, then open `http://127.0.0.1:8050/` in your browser.

In [ ]:
!zenml integration install dash -f

In [ ]:
digits_svc_pipeline = digits_pipeline(
    importer=importer(), trainer=svc_trainer(), evaluator=evaluator()
)
digits_svc_pipeline.run()

In [ ]:
from zenml.integrations.dash.visualizers.pipeline_run_lineage_visualizer import (
    PipelineRunLineageVisualizer,
)
from zenml.repository import Repository

repo = Repository()
latest_run = repo.get_pipeline("digits_pipeline").runs[-1]
PipelineRunLineageVisualizer().visualize(latest_run)

You should now see an interactive visualization in your browser as shown below. Squares represent your artifacts, circles your pipeline steps. Also note that the different nodes are color coded, so if your pipeline ever fails or runs for too long, you can find the responsible step at a glance!

![Dash Visualization](_assets/02_Artifact_Lineage/dash_initial.png)

## Artifact Caching
As mentioned in the beginning, tracking which exact artifact went into what steps also allows us to cache and reuse artifacts. Let's see this in action:
First, stop the execution of the previous notebook cell in case it is still running. Then, execute the next cell to rerun our pipeline and visualize it with dash again.

In [ ]:
digits_svc_pipeline.run()
latest_run = repo.get_pipeline("digits_pipeline").runs[-1]
PipelineRunLineageVisualizer().visualize(latest_run)

You should now see a visualization as shown below. Note that the color of all nodes in the graph changed to green now. This means they were still cached from our previous run.

![Dash Visualization Cached](_assets/02_Artifact_Lineage/dash_cached.png)

Let's now replace the SVC model in our ML pipeline with a decision tree and see what happens.

In [ ]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.tree import DecisionTreeClassifier
from zenml.steps import step


@step()
def tree_trainer(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
    """Train a sklearn decision tree classifier."""
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    return model


# redefine and rerun our pipeline, this time with tree_trainer()
digits_tree_pipeline = digits_pipeline(
    importer=importer(), trainer=tree_trainer(), evaluator=evaluator()
)
digits_tree_pipeline.run()

latest_run = repo.get_pipeline("digits_pipeline").runs[-1]
PipelineRunLineageVisualizer().visualize(latest_run)

The visualization should now look as shown below. Since we changed the trainer, the corresponding node and all subsequent nodes are now blue again, meaning they were rerun and the artifacts were freshly created. However, note how the input data artifacts are still green. They did not have to be recreated. In a real production setting this might save us a tremendous amount of time and resources as those data artifacts might have been the result of some complex, expensive preprocessing job.

![Dash Visualization Partly Cached](_assets/02_Artifact_Lineage/dash_partly_cached.png)
